In [ ]:
import numpy as np
import pandas as pd
import igraph as ig
import vmlab

In [ ]:
from vmlab.models import fruit_model
from vmlab.processes import harvest
fruit_model = fruit_model.update_processes({'harvest': harvest.HarvestByQuality})
fruit_model.visualize()

In [ ]:
tree = pd.DataFrame({
    'parent_id': [np.nan, 0, 0],
    'id': [0, 1, 2],
    'cycle': [3, 3, 3],
    'is_apical': [1, 0, 1],
    'appearance_month': [2, 2, 2],
    'ancestor_nature': [0, 0, 0],
    'ancestor_is_apical': [1, 1, 1],
    'nature': [0, 0, 0],
    'burst_date': ['NAT', 'NAT', 'NAT'],
    'flowering_date': ['NAT', '2002-10-01', '2002-09-01'],
    'has_apical_child': [1, 0, 0],
    'nb_lateral_children': [1, 0, 0],
    'nb_inflo': [0, 1, 2],
    'nb_fruit': [0, 1, 1]
    
})
in_g = ig.Graph.DataFrame(tree.dropna())
ig.plot(in_g, bbox=(0,0,150,150), layout=in_g.layout_reingold_tilford())

In [ ]:
setup = vmlab.create_setup(
    model=fruit_model,
    tree=tree,
    start_date='2002-06-01',
    end_date='2003-06-01',
    setup_toml='../share/setup/fruit_model.toml',
    current_cycle=3,
    input_vars={
        'topology': {
            'seed': 11,
        },
        'geometry__interpretation_freq': 15
    },
    output_vars={
        'topology': {
            'adjacency': 'day'
        },
        'environment': {
            'TM_day': 'day',
            'RH_day': 'day'
        },
        'carbon_allocation': {
            'DM_fruit': 'day',
            'DM_fleshpeel': 'day'
        },
        'harvest': {
            'ripeness_index': 'day',
            'nb_fruit_harvested': None
        },
        'growth': {
            'nb_leaf': None,
        },
        'fruit_quality': {
            'FM_fruit': 'day',
            'W_fleshpeel': 'day',
            'DM_flesh': 'day',
            'W_flesh': 'day',
            'transpiration_fruit': 'day',
            'osmotic_pressure_fruit': 'day',
            'turgor_pressure_fruit': 'day',
            'water_potential_fruit': 'day',
            'flux_xylem_phloem': 'day',
            'sucrose': 'day'
        }
    }
)

In [ ]:
ds_out = vmlab.run(setup, fruit_model, geometry=True)

In [ ]:
g = ig.Graph.Adjacency([row.tolist() for row in ds_out.topology__adjacency[-1].data.astype(np.int64)])
layout = g.layout_reingold_tilford(root=0.)
layout.rotate(-180)
ig.plot(g, layout=layout, bbox=(600, 300), **{
    'vertex_size': 1,
    'vertex_label_size': 10,
    'edge_arrow_width': 0.1,
    'vertex_label': [
        f'GU{idx}\nF:{int(ds_out.harvest__nb_fruit_harvested.data[idx])}\nL:{int(ds_out.growth__nb_leaf.data[idx])}' for idx in g.vs.indices
    ]
})

In [ ]:
ds_out.carbon_allocation__DM_fleshpeel.plot(col='GU', col_wrap=6)

In [ ]:
ds_out.fruit_quality__DM_flesh.plot(col='GU', col_wrap=6)

In [ ]:
ds_out.fruit_quality__FM_fruit.plot(col='GU', col_wrap=6)

In [ ]:
ds_out.fruit_quality__W_fleshpeel.plot(col='GU', col_wrap=6)

In [ ]:
ds_out.fruit_quality__flux_xylem_phloem.plot(col='GU', col_wrap=6)

In [ ]:
ds_out.fruit_quality__turgor_pressure_fruit.plot(col='GU', col_wrap=6)

In [ ]:
ds_out.fruit_quality__transpiration_fruit.plot(col='GU', col_wrap=6)

In [ ]:
ds_out.environment__TM_day.sel({'day': slice('2002-11-01', '2003-05-01')}).plot(col_wrap=6)

In [ ]:
ds_out.environment__RH_day.sel({'day': slice('2002-11-01', '2003-05-01')}).plot(col_wrap=6)